In [56]:
import pandas as pd 
import numpy as np
import pickle
import os
import glob


root = 'C:\\Users\\User\\0. 노션 포트폴리오\\개인 배차최적화'
root = root.replace('\\','/')

with open(root + '/datas/dist_matrix.pkl', 'rb') as f:
    dist = pickle.load(f)
    
with open(root + '/datas/time_matrix.pkl', 'rb') as f:
    time = pickle.load(f)
    
hub = pd.read_excel(root + '/datas/hubs.xlsx')

In [57]:
dist

array([[  0.    ,  31.2863,  15.3286, ...,   2.5561, 330.2813,  16.5199],
       [ 31.3479,   0.    ,  22.0772, ...,  31.1674, 347.8071,  18.0099],
       [ 15.6206,  21.9734,   0.    , ...,  13.6262, 339.5899,  10.0246],
       ...,
       [  2.5561,  31.3773,  13.3341, ...,   0.    , 333.5089,  14.4927],
       [330.1992, 347.7323, 339.2413, ..., 333.4291,   0.    , 332.9659],
       [ 16.5417,  17.9543,  10.0665, ...,  14.3477, 333.0009,   0.    ]],
      shape=(49, 49))

In [65]:
os.getcwd()

'C:\\Users\\User\\0. 노션 포트폴리오\\개인 배차최적화\\scripts'

In [66]:
p = glob.glob("C:\\Users\\User\\0. 노션 포트폴리오\\개인 배차최적화\\compare_outputs\\*.csv")
df = pd.read_csv(*p)

In [67]:
df

,run_id,version,time_limit_sec,drop_penalty,frozen_dry_penalty,vehicle_fixed_cost,wait_cost_coeff,dispatch_rate,total_km,total_time_min,travel_min,wait_min,wait_rate,used_vehicles,dropped,objective
0,0,before,10,10000000,10000000,240000,200,0.98,2099.2437,1944,1879,65,0.033436,5,1,13700204
1,0,after,10,10000000,10000000,556058,0,0.98,2007.2647,2141,1790,351,0.163942,4,1,14231458
2,1,before,10,10000000,10000000,240000,200,0.94,1448.8523,1298,1290,8,0.006163,4,3,32668413
3,1,after,10,10000000,10000000,556058,0,0.94,1270.5006,1613,1141,472,0.292622,4,3,33494688
4,2,before,10,10000000,10000000,240000,200,0.98,1474.4545,1450,1320,130,0.089655,4,1,12727415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,47,after,10,10000000,10000000,556058,0,0.94,1680.1110,1845,1523,322,0.174526,4,3,33904308
96,48,before,10,10000000,10000000,240000,200,0.94,3056.6100,2606,2598,8,0.003070,5,3,34777973
97,48,after,10,10000000,10000000,556058,0,0.94,3135.7882,3246,2704,542,0.166975,6,3,36472095
98,49,before,10,10000000,10000000,240000,200,0.92,1127.1379,1055,1029,26,0.024645,3,4,42060707


In [68]:
list(df.columns[7:])

['dispatch_rate',
 'total_km',
 'total_time_min',
 'travel_min',
 'wait_min',
 'wait_rate',
 'used_vehicles',
 'dropped',
 'objective']

In [69]:
df.groupby("version").agg({'dispatch_rate':'mean',
                          'total_km':'mean',
                          'travel_min':'mean',
                          'wait_rate':'mean',
                          'objective':'mean',
                          'used_vehicles':'mean'})

,dispatch_rate,total_km,travel_min,wait_rate,objective,used_vehicles
version,,,,,,
after,0.9552,2316.272414,2001.16,0.163752,27374189.96,4.78
before,0.9552,2343.916410,2022.92,0.036003,26338209.38,4.86


In [55]:
import os
import pickle
import numpy as np

def _percentiles(x, ps=(1, 5, 10, 25, 50, 75, 90, 95, 99)):
    x = np.asarray(x, dtype=float)
    x = x[np.isfinite(x)]
    if x.size == 0:
        return {p: np.nan for p in ps}
    return {p: float(np.percentile(x, p)) for p in ps}

def analyze_dist_matrix(dist, depot=0):
    """
    dist_matrix: (H x H) in km
    - depot->others 분포
    - 전체 hub->hub(대각/0 제외) 분포
    """
    dist = np.asarray(dist, dtype=float)

    # depot -> others (0 제외)
    d0 = dist[depot].copy()
    d0 = d0[(d0 > 0) & np.isfinite(d0)]

    # 전체 (대각/0 제외)
    allv = dist.reshape(-1)
    allv = allv[(allv > 0) & np.isfinite(allv)]

    stats = {
        "depot_min_km": float(np.min(d0)) if d0.size else np.nan,
        "depot_pctl_km": _percentiles(d0),
        "all_pctl_km": _percentiles(allv),
        "n_depot_edges": int(d0.size),
        "n_all_edges": int(allv.size),
    }
    return stats

def suggest_vehicle_fixed_cost_km(stats, base="all_p75", km_range=(50, 800)):
    """
    OR-Tools에서 dist objective는 km*1000 => '미터 단위 정수 비용'으로 들어가니까,
    vehicle_fixed_cost도 (K km) * 1000 으로 맞추면 됨.
    - km_range: '차량 1대 = K km' 가정의 탐색 범위
    """
    # 대표값 참고용(선택)
    depot_p75 = stats["depot_pctl_km"].get(75, np.nan)
    all_p75 = stats["all_pctl_km"].get(75, np.nan)

    print("=== DIST STATS (km) ===")
    print(f"depot->others  min: {stats['depot_min_km']:.4f} km")
    print(f"depot->others  p50: {stats['depot_pctl_km'][50]:.4f} km | p75: {depot_p75:.4f} km | p90: {stats['depot_pctl_km'][90]:.4f} km")
    print(f"all hub->hub   p50: {stats['all_pctl_km'][50]:.4f} km | p75: {all_p75:.4f} km | p90: {stats['all_pctl_km'][90]:.4f} km")
    print()

    lo_km, hi_km = km_range
    lo_cost = int(lo_km * 1000)   # km -> meter-cost
    hi_cost = int(hi_km * 1000)

    print("=== RECOMMENDED SEARCH SPACE ===")
    print(f"[vehicle_fixed_cost]  {lo_cost:,} ~ {hi_cost:,}  (차량 1대 = {lo_km}~{hi_km} km 가정)")
    print(f"[wait_cost_coeff]     0 ~ 2000  (너무 키우면 차량 늘어나는 쪽으로 튈 수 있음)")
    print("(drop_penalty / frozen_dry_penalty는 정책값으로 고정 추천)")
    return lo_cost, hi_cost

if __name__ == "__main__":

    stats = analyze_dist_matrix(dist=dist, depot=0)
    suggest_vehicle_fixed_cost_km(stats, km_range=(50, 800))


=== DIST STATS (km) ===
depot->others  min: 1.4852 km
depot->others  p50: 16.5658 km | p75: 27.8971 km | p90: 60.4424 km
all hub->hub   p50: 15.8994 km | p75: 43.8475 km | p90: 73.0274 km

=== RECOMMENDED SEARCH SPACE ===
[vehicle_fixed_cost]  50,000 ~ 800,000  (차량 1대 = 50~800 km 가정)
[wait_cost_coeff]     0 ~ 2000  (너무 키우면 차량 늘어나는 쪽으로 튈 수 있음)
(drop_penalty / frozen_dry_penalty는 정책값으로 고정 추천)


In [ ]:
# -----------------------------
# 실행
# -----------------------------
# & "C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\Scripts\optuna-dashboard.exe" sqlite:///optuna.db 

In [102]:
# hub - hub 최대 소요 시간

np.sum(time.max())

299.2566666666667

In [109]:
tm = time.copy()
np.fill_diagonal(tm, np.nan)

p50 = np.nanpercentile(tm, 50)
p90 = np.nanpercentile(tm, 90)
p95 = np.nanpercentile(tm, 95)
p99 = np.nanpercentile(tm, 99)
mx  = np.nanmax(tm)

p50, p90, p95, p99, mx

(16.93, 55.877833333333335, 70.09883333333327, 278.1765, 299.2566666666667)